<a href="https://colab.research.google.com/github/Veena7580/Homework/blob/main/Weather_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make a weather app that:
Let users enter a location (any of the following:  Zip Code/Postal Code, GPS Coordinates, Landmarks, Town, City, etc..)  and get the current weather.  You have autonomy to determine how to handle how the user should enter their location
Shows the weather clearly, with any details you think are useful.
Let users see the weather based on their current location.
Use icons or images to make the weather info look cool using any design standards.


In [ ]:
pip install sqlalchemy sqlite

ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define the base class for SQLAlchemy models
Base = declarative_base()

# Define the Location model (table)
class Location(Base):
    __tablename__ = 'locations'

    id = Column(Integer, primary_key=True)
    key = Column(String, unique=True)
    name = Column(String)
    status = Column(String)

# Create an SQLite database (or replace with your database URL)
DATABASE_URL = "sqlite:///locations.db"
engine = create_engine(DATABASE_URL, echo=True)

# Create the database tables if they don't exist
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Insert a record for testing
def add_sample_data():
    # Sample record for "Saint Louis"
    sample_location = Location(
        key='9c7781a942e6115d80f8b530fefe9b85',  # Unique key
        name='Saint Louis',
        status='Active'
    )
    session.add(sample_location)
    session.commit()

# Function to query and get the location by name
def get_location_by_name(name):
    location = session.query(Location).filter(Location.name == name).first()
    return location

# Function to update the status of a location
def update_location_status(name, new_status):
    location = session.query(Location).filter(Location.name == name).first()
    if location:
        location.status = new_status
        session.commit()
        return f"Status of {name} updated to {new_status}"
    else:
        return f"Location {name} not found."

# Uncomment this to add sample data (run once)
# add_sample_data()

# Example: Get "Saint Louis" location
location = get_location_by_name("Saint Louis")
if location:
    print(f"Location: {location.name}, Status: {location.status}")
else:
    print("Location not found.")

# Example: Update status of "Saint Louis" to "Inactive"
update_message = update_location_status("Saint Louis", "Inactive")
print(update_message)

# Confirm the update by getting the location again
location = get_location_by_name("Saint Louis")
if location:
    print(f"Updated Location: {location.name}, Status: {location.status}")


2025-04-04 23:21:13,874 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-14-6f0c001b3eff>:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-04-04 23:21:13,880 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("locations")


2025-04-04 23:21:13,884 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:[cached since 0.01393s ago] ('Saint Louis', 1, 0)


Location Saint Louis not found.
2025-04-04 23:21:13,958 INFO sqlalchemy.engine.Engine SELECT locations.id AS locations_id, locations."key" AS locations_key, locations.name AS locations_name, locations.status AS locations_status 
FROM locations 
WHERE locations.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT locations.id AS locations_id, locations."key" AS locations_key, locations.name AS locations_name, locations.status AS locations_status 
FROM locations 
WHERE locations.name = ?
 LIMIT ? OFFSET ?


2025-04-04 23:21:13,961 INFO sqlalchemy.engine.Engine [cached since 0.02187s ago] ('Saint Louis', 1, 0)


INFO:sqlalchemy.engine.Engine:[cached since 0.02187s ago] ('Saint Louis', 1, 0)


In [ ]:
pip install flask requests

In [ ]:
from flask import Flask, render_template
import requests

# Initialize Flask app
app = Flask(__name__)

# API key for weather API (replace with your actual key)
API_KEY = "your_weather_api_key"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Function to get weather data from OpenWeatherMap API
def get_weather(city_name):
    params = {
        'q': city_name,
        'appid': API_KEY,
        'units': 'metric'  # For Celsius temperature, you can change this to 'imperial' for Fahrenheit
    }
    response = requests.get(BASE_URL, params=params)
    return response.json()

# Route for the dashboard
@app.route('/')
def dashboard():
    # Get the location from the database
    location = get_location_by_name("Saint Louis")
    if location:
        weather_data = get_weather(location.name)
        if weather_data.get("cod") == 200:
            main_data = weather_data["main"]
            weather_info = weather_data["weather"][0]
            return render_template("dashboard.html", location=location, main_data=main_data, weather_info=weather_info)
        else:
            return render_template("dashboard.html", error="City not found.")
    else:
        return "Location not found in database."

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
